In [ ]:
import os
os.chdir('..')
import reproducibility

# Inside Neural Networks for Fuel Moisture
## Jan Mandel, University of Colorado Denver


## Building and evaluating RNN

A recurrent neural network (RNN) has a similar information flow but it can be more flexible and look for the best model automatically, i.e., build the model from data. 

We'll start by how to evaluate the map, then actually create it later.

The following is based on https://machinelearningmastery.com/understanding-simple-recurrent-neural-networks-in-keras/


In [ ]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from keras.utils.vis_utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt
import tensorflow as tf
import keras.backend as K

In [ ]:
def create_RNN(hidden_units, dense_units, input_shape, activation):
    inputs = tf.keras.Input(shape=input_shape)
    # https://stackoverflow.com/questions/43448029/how-can-i-print-the-values-of-keras-tensors
    # inputs2 = K.print_tensor(inputs, message='inputs = ')  # change allso inputs to inputs2 below, must be used
    x = tf.keras.layers.SimpleRNN(hidden_units, input_shape=input_shape,
                        activation=activation[0])(inputs)
    outputs = tf.keras.layers.Dense(dense_units, activation=activation[1])(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
# Demo example
hidden=5
features=2
timesteps=3
demo_model = create_RNN(hidden_units=hidden, dense_units=1, 
                        input_shape=(timesteps,features), 
                        activation=['linear', 'linear'])
print(demo_model.summary())
w = demo_model.get_weights()
#print(len(w),' weight arrays:',w)
wname=('wx','wh','bh','wy','by','wz','bz')
for i in range(len(w)):
  print(i,':',wname[i],'shape=',w[i].shape)
wx, wh, bh, wy, by = w
plot_model(demo_model, to_file='model_plot.png', 
  show_shapes=True, show_layer_names=True,
  expand_nested=True,)

The input layer here is just a formality. The input of the hidden layer `simple_rnn` consist of vector passed by the input layer, followed by its own output from the previous time step.

Now let’s do a simple experiment to see how the layers from a SimpleRNN and Dense layer produce an output. Keep this figure in view.
<img src="https://machinelearningmastery.com/wp-content/uploads/2021/09/rnnCode1.png">

We’ll input x for three time steps and let the network generate an output. The values of the hidden units at time steps 1, 2 and 3 will be computed. $h(0)$ is initialized to the zero vector. The output $o(3)$ is computed from $h(3)$ and $w(3)$. An activation function is linear, $f(x)=x$, so the update of  $h(k)$  and the output $o(k)$ are given by
\begin{align*}
h\left(  0\right)  = &0  \\
h\left(  k+1\right)  =& 
x\left(  k\right) w_{x}
  +h(k) w_{h}  + b_{h}\\
o(k+1)=& h(k+1)w_{y} + b_y
\end{align*}

In [ ]:
# Reshape the input to sample_size x time_steps x features 
samples=4   # number of samples
x = tf.reshape(tf.range(samples*timesteps*features),[samples,timesteps,features]) 
print('test input x=',x)
print('model.predict start')
y_pred_model = demo_model.predict(x)
print('model.predict end')

outputs = 1
o3=np.zeros([samples,outputs])
h_3 = np.zeros(hidden)
for i in range(samples):
  h_0 = np.zeros(hidden)
  h_1 = np.dot(x[i,0,:], wx) + np.dot(h_0,wh) + bh
  print('h_1=',h_1)
  h_2 = np.dot(x[i,1,:], wx) + np.dot(h_1,wh) + bh
  h_3 = np.dot(x[i,2,:], wx) + np.dot(h_2,wh) + bh
  o3[i,:] = np.dot(h_3, wy) + by
# in coordinates
h = np.zeros(hidden)
o = np.zeros([samples,outputs])
for i in range(samples):
    for j in range(timesteps):
        hout = np.zeros(hidden)
        for k in range(hidden):
            hout[k] = bh[k]
            for l in range(features):
                hout[k] += x[i,j,l]*wx[l,k] 
            for l in range(hidden):
                hout[k] += h[l]*wh[l,k]
        h = hout.copy()
        print('timestep=',k,'h=',h)
    for l in range(outputs):
        o[i,:] = by
        for k in range(hidden):
            # print('i=',i,'k=',k,'l=',l)
            o[i,l] += h[k]*wy[k,l]

print("Prediction from network ", y_pred_model)
print("Prediction from our computation ", o3)
print("Prediction computed by loops ", o)

The result is the same.

In [ ]:
input("Press Enter to continue...")

## Fuel moisture models




### A simple fuel moisture model

First consider a simplified fuel moisture model without considering the effect of rain.
The evolution of fuel moisture content $m(t)$ is modeled by the time-lag differential equation on interval $\left[
t_{0},t_{1}\right]  $,
$$
\frac{dm}{dt}=\frac{E-m(t)}{T},\quad m(t_{0})=m_{0}.
$$
where the initial fuel moisture content $m_{0}=m\left(  t_{0}\right)  $ is the
input, and $m_{1}=m(t_{1})$ is the output. Tnus, $m_1=F(m_0)$. The parameters of the model are the
fuel moisture equilibrium $E$, assumed to be constant over the interval $\left[
t_{0},t_{1}\right]  $, NS the characteristic decay time $T$. 

We can build the general model later by calling this simple model with different
equilibria and time constants (drying, wetting, rain).

Since $E$ is constant in time, the solution can be found
analytically,
$$
m\left(  t\right)  =E+\left(  m_{0}-E\right)  e^{-t/T}%
$$
For convenience, we use $T_{1}=1/T$ instead of $T$, and the model becomes
$$
m_{1}=E+\left(  m_{0}-E\right)  e^{-\left(  t_{1}-t_{0}\right)  T_{1}}%
$$
In the extended Kalman filter, we will need the partial derivatives of $m_{1}$
with respect to the input and the parameters. Compute
$$
\frac{dm_{1}}{d_{m0}}=e^{-\left(  t_{1}-t_{0}\right)  T_{1}}
$$
$$
\frac{dm_{1}}{dE}=1-e^{-\left(  t_{1}-t_{0}\right)  T_{1}}
$$
$$
\frac{dm_{1}}{dT_{1}}=-\left(  m_{0}-E\right)  \left(  t_{1}-t_{0}\right)
e^{-\left(  t_{1}-t_{0}\right)  T_{1}}
$$
At the moment, we need only ${dm_{1}}/{dm_{0}}$ but we put in the code all partials for possible use in future.


In [ ]:
import numpy as np
def model_decay(m0,E,partials=0,T1=0.1,tlen=1):  
  # Arguments: 
  #   m0          fuel moisture content at start dimensionless, unit (1)
  #   E           fuel moisture eqilibrium (1)
  #   partials=0: return m1 = fuel moisture contents after time tlen (1)
  #           =1: return m1, dm0/dm0 
  #           =2: return m1, dm1/dm0, dm1/dE
  #           =3: return m1, dm1/dm0, dm1/dE dm1/dT1   
  #   T1          1/T, where T is the time constant approaching the equilibrium
  #               default 0.1/hour
  #   tlen        the time interval length, default 1 hour

  exp_t = np.exp(-tlen*T1)                  # compute this subexpression only once
  m1 = E + (m0 - E)*exp_t                   # the solution at end
  if partials==0:
    return m1
  dm1_dm0 = exp_t
  if partials==1:
    return m1, dm1_dm0          # return value and Jacobian
  dm1_dE = 1 - exp_t      
  if partials==2:
     return m1, dm1_dm0, dm1_dE 
  dm1_dT1 = -(m0 - E)*tlen*exp_t            # partial derivative dm1 / dT1
  if partials==3:
    return m1, dm1_dm0, dm1_dE, dm1_dT1       # return value and all partial derivatives wrt m1 and parameters
  raise('Bad arg partials')
  

### Fuel moisture model with drying equilibrium, wetting equilibrium, and rain

Here is a little more realistic fuel moisture model from Mandel et al. (2004). A rain-wetting lag time $t_{\mathrm{r}}$ is reached for heavy rain only
asymptotically, when the rain intensity $r$ (mm/h) is
large:
$$
\frac{\mathrm{d}m}{\mathrm{d}t}=\frac{S-m}{t_{\mathrm{r}}}\left(1-\exp\left(-\frac{r-r_0}{r_{\mathrm{s}}}
\right)  \right),\ \text{if}\ r>r_0, 
$$
where $r_0$ is the threshold rain intensity below which no perceptible
wetting occurs, and $r_{\mathrm{s}}$ is the saturation rain
intensity. At the saturation rain intensity, $1-1/e\approx 0.63$ of
the maximal rain-wetting rate is achieved. For 10h fuel, the model takes $S=250\,{\%}$,
$t_{\mathrm{r}}=14$h, $r_0=0.05$mm/h and
$r_{\mathrm{s}}=8$mm/h. 

In [ ]:
### Define model function with drying, wetting, and rain equilibria

# Parameters
r0 = 0.05                                   # threshold rainfall [mm/h]
rs = 8.0                                    # saturation rain intensity [mm/h]
Tr = 14.0                                   # time constant for rain wetting model [h]
S = 250                                     # saturation intensity [dimensionless]
T = 10.0                                    # time constant for wetting/drying

def model_moisture(m0,Eqd,Eqw,r,t,partials=0,T=10.0,tlen=1.0):
    # arguments:
    # m0         starting fuel moistureb (%s
    # Eqd        drying equilibrium      (%) 
    # Eqw        wetting equilibrium     (%)
    # r          rain intensity          (mm/h)
    # t          time
    # partials = 0, 1, 2
    # returns: same as model_decay
    #   if partials==0: m1 = fuel moisture contents after time 1 hour
    #              ==1: m1, dm1/dm0 
    #              ==2: m1, dm1/dm0, dm1/dE  
    
    if r > r0:
        # print('raining')
        E = S
        T1 =  (1.0 - np.exp(- (r - r0) / rs)) / Tr
    elif m0 <= Eqw: 
        # print('wetting')
        E=Eqw
        T1 = 1.0/T
    elif m0 >= Eqd:
        # print('drying')
        E=Eqd
        T1 = 1.0/T
    else: # no change'
        E = m0
        T1=0.0
    exp_t = np.exp(-tlen*T1)
    m1 = E + (m0 - E)*exp_t  
    dm1_dm0 = exp_t
    dm1_dE = 1 - exp_t
    #if t>=933 and t < 940:
    #  print('t,Eqw,Eqd,r,T1,E,m0,m1,dm1_dm0,dm1_dE',
    #        t,Eqw,Eqd,r,T1,E,m0,m1,dm1_dm0,dm1_dE)   
    if partials==0: 
        return m1
    if partials==1:
        return m1, dm1_dm0
    if partials==2:
        return m1, dm1_dm0, dm1_dE
    raise('bad partials')

#### Training and forecasting with the RNN

We are given a sequence `x` of inputs size `[train_steps+forecast_steps,features]` and want to train a model so that at step `i` in `range(train_steps)`, the model returns close to `features[i,:]`. The trained model then returns for `i` in `range(train_steps,train_steps+forecast_steps)` a forecast `features[i,:]`.

In [ ]:
def staircase(x,y,timesteps,trainsteps,return_sequences=False):
  # x [trainsteps+forecaststeps,features]    all inputs
  # y [trainsteps,outputs]
  # timesteps: split x and y into samples length timesteps, shifted by 1
  # trainsteps: number of timesteps to use for training, no more than y.shape[0]
  print('shape x = ',x.shape)
  print('shape y = ',y.shape)
  print('timesteps=',timesteps)
  print('trainsteps=',trainsteps)
  outputs = y.shape[1]
  features = x.shape[1]
  forecaststeps = x.shape[0]-trainsteps
  samples = trainsteps-timesteps+1
  print('staircase: samples=',samples,'timesteps=',timesteps,'features=',features)
  x_train = np.empty([samples, timesteps, features])
  print('return_sequences=',return_sequences)
  if return_sequences:
    print('returning all timesteps in a sample')
    y_train = np.empty([samples, timesteps, outputs])  # all
    for i in range(samples):
      for k in range(timesteps):
        for j in range(features):
          x_train[i,k,j] = x[i+k,j]
        for j in range(outputs):
          y_train[i,k,j] = y[i+k,j]
  else:
    print('returning only the last timestep in a sample')
    y_train = np.empty([samples, outputs])
    for i in range(samples):
      for j in range(features):
        for k in range(timesteps):
          x_train[i,k,j] = x[i+k,j]
      for j in range(outputs):
        y_train[i,j] = y[i+timesteps-1,j]

  return x_train, y_train

In [ ]:
def seq2batches(x,y,timesteps,trainsteps):
  # x [trainsteps+forecaststeps,features]    all inputs
  # y [trainsteps,outputs]
  # timesteps: split x and y into samples length timesteps, shifted by 1
  # trainsteps: number of timesteps to use for training, no more than y.shape[0]
  print('shape x = ',x.shape)
  print('shape y = ',y.shape)
  print('timesteps=',timesteps)
  print('trainsteps=',trainsteps)
  outputs = y.shape[1]
  features = x.shape[1]
  samples= trainsteps - timesteps + 1
  print('samples=',samples)
  x_train = np.empty([samples, timesteps, features])
  y_train = np.empty([samples, timesteps, outputs])  # only the last
  print('samples=',samples,' timesteps=',timesteps,
        ' features=',features,' outputs=',outputs)
  for i in range(samples):
    for k in range(timesteps):
      for j in range(features):
        x_train[i,k,j] = x[i+k,j]
      for j in range(outputs):
        y_train[i,k,j] = y[i+k,j]  # return sequences
  return x_train, y_train

In [ ]:
print('test preprocessing for RNN')
trainsteps=5
features=1
outputs=1
timesteps=3
x = tf.reshape(tf.range(trainsteps*features),[trainsteps,features])
y = tf.reshape(tf.range(trainsteps*outputs),[trainsteps,outputs])
print('x=',x)
print('y=',y)
x_train, y_train = staircase(x,y,timesteps,trainsteps)
print('x_train=',x_train)
print('y_train=',y_train)
x_train, y_train = seq2batches(x,y,timesteps,trainsteps)
print('x_train=',x_train)
print('y_train=',y_train)

In [ ]:
E,m_f,data,hour,h2,DeltaE = create_synthetic_data(days=20,power=4,data_noise=0.01,process_noise=0.0,DeltaE=0.1) 

In [ ]:
scale=False
# transform as 2D, (timesteps, features) and (timesteps, outputs)
Et = np.reshape(E,[E.shape[0],1])
datat = np.reshape(data,[data.shape[0],1])
if scale:
  scalerx = MinMaxScaler()
  scalerx.fit(Et)
  Et = scalerx.transform(Et)
  scalery = MinMaxScaler()
  scalery.fit(datat)
  datat = scalery.transform(datat)

In [ ]:
def create_RNN_2(hidden_units, dense_units, activation, stateful=False, 
                 batch_shape=None, input_shape=None, dense_layers=1,
                 rnn_layers=1,return_sequences=False,
                 initial_state=None):
    if stateful:
      inputs = tf.keras.Input(batch_shape=batch_shape)
    else:
      inputs = tf.keras.Input(shape=input_shape)
    # https://stackoverflow.com/questions/43448029/how-can-i-print-the-values-of-keras-tensors
    # inputs2 = K.print_tensor(inputs, message='inputs = ')  # change allso inputs to inputs2 below, must be used
    x = inputs
    for i in range(rnn_layers):
      x = tf.keras.layers.SimpleRNN(hidden_units,activation=activation[0],
              stateful=stateful,return_sequences=return_sequences)(x
              # ,initial_state=initial_state
              )
    # x = tf.keras.layers.Dense(hidden_units, activation=activation[1])(x)
    for i in range(dense_layers):
      x = tf.keras.layers.Dense(dense_units, activation=activation[1])(x)
    model = tf.keras.Model(inputs=inputs, outputs=x)
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
# split data
return_sequences=False
shift = 0.
print('shifting inputs by',shift)
x_train, y_train = staircase(Et+shift,datat+shift,timesteps=5,trainsteps=h2,
                             return_sequences=return_sequences)
print('x_train shape=',x_train.shape)
samples, timesteps, features = x_train.shape
print('y_train shape=',y_train.shape)
# the simplest model possible
activation=['linear','linear']
hidden_units=1
dense_units=1
dense_layers=1
features=1
hours=Et.shape[0]
h0 = tf.convert_to_tensor(datat[:samples],dtype=tf.float32)
# print('initial state=',h0)
# statefull model version for traning
import moisture_rnn
# model_fit=create_RNN_2(hidden_units=hidden_units, 
model_fit=moisture_rnn.create_RNN_2(hidden_units=hidden_units, 
                        dense_units=dense_units, 
                        batch_shape=(samples,timesteps,features),
                        stateful=True,
                        return_sequences=return_sequences,
                        # initial_state=h0,
                        activation=activation,
                        dense_layers=dense_layers)
# same model stateless for prediction on the entire dataset - to start onlg
# the real application will switch to prediction after training data end
# and start from the state there
print('model_fit input shape',x_train.shape,'output shape',model_fit(x_train).shape)
from keras.utils.vis_utils import plot_model
plot_model(model_fit, to_file='model_plot.png', 
           show_shapes=True, show_layer_names=True)

In [ ]:
model_predict=create_RNN_2(hidden_units=hidden_units, dense_units=dense_units,  
                        input_shape=(hours,features),stateful = False,
                        return_sequences=True,
                        activation=activation,dense_layers=dense_layers)
# model_predict=create_RNN_sequences(hidden_units=1, dense_units=1, input_shape=(hours,1), 
#                        activation=['linear', 'linear'])
print('model_predict input shape',Et.shape,'output shape',model_predict(Et).shape)
print(model_predict.summary())
from keras.utils.vis_utils import plot_model
plot_model(model_predict, to_file='model_plot.png', 
           show_shapes=True, show_layer_names=True)

In [ ]:
# fitting
w_exact=  [np.array([[1.-np.exp(-0.1)]]), np.array([[np.exp(-0.1)]]), np.array([0.]),np.array([[1.0]]),np.array([-1.*DeltaE])]
w_initial=[np.array([[1.-np.exp(-0.1)]]), np.array([[np.exp(-0.1)]]), np.array([0.]),np.array([[1.0]]),np.array([0.*DeltaE])]
model_fit.set_weights(w_initial)
model_fit.fit(x_train, y_train, epochs=1000, verbose=0,batch_size=samples)
w_fitted=model_fit.get_weights()
for i in range(len(w)):
  print('weight',i,' exact:',w_exact[i],':  initial:',w_initial[i],' fitted:',w_fitted[i])

In [ ]:
def model_eval(w,title):
  # prediction on the entire dataset from zero state
  model_predict.set_weights(w)
  hours=Et.shape[0]
  print('Et.shape=',Et.shape,'hours=',hours)
  x_input=np.reshape(Et,(1, hours, 1))
  y_output = model_predict.predict(x_input)
  print('x_input.shape=',x_input.shape,'y_output.shape=',y_output.shape)
  m = np.reshape(y_output,hours) - shift
  print('weights=',w)
  if scale:
    print('scaling')
    m = scalery.inverse_transform(m)
  m = np.reshape(m,hours)
  plot_m(m,title=title)
  return m

In [ ]:
m_fitted=model_eval(w_fitted,'RNN prediction with fitted weights')

In [ ]:
m_exact=model_eval(w_exact,'RNN prediction with exact weights')

In [ ]:
m_initial=model_eval(w_initial,'RNN prediction with initial weights')

In [ ]:
out = np.empty((hours,1))
w=w_exact
h=0
for i in range(Et.shape[0]):
  h=np.dot(Et[i,0],w[0])+np.dot(h,w[1]) + w[2]
  out[i]=np.dot(h,w[3]) + w[4]
if scale:
  print('scaling')
  out = scalery.inverse_transform(out)
out=np.reshape(out,hours)
print('max abs diff',np.max(np.abs(m_exact-out)))
plot_m(out,title='Hand computed RNN prediction with exact weights')

### 3.2 Acquisition and preprocessing of real data

Data assimilation for fuel moisture from Remote Automated Weather Stations (RAWS) was developed in Vejmelka et al. (2016). First, they use regression from all RAWS in a given area to extend the data spatially from RAWS to a grid in the whole area, then they run the extended Kalman filter at each grid node. Here, we are interested in a simplified problem: estimate future fuel moisture at a single RAWS location from weather data.  

#### 3.2.1 Acquisition of fuel moisture observations

We try to load the data from a saved file first. If that fails, retrieve the fuel moisture data from sensors on weather stations in the Mesowest network. Get all stations with fuel moisture data in a spatial box within one hour, then pick one station and retrieve the whole time series.

In [ ]:
import json
os.chdir('data')

In [ ]:
jfile = 'raws.json'; vars='fuel_moisture'; case = 1
# jfile = 'raws2.json'; vars='fuel_moisture,precip_accum_one_hour'; case = 2
def json_w(j,f):
  print('writing json file',f)
  json.dump(j,open(f,'w'),indent=4)
try:
    #! wget --no-clobber http://math.ucdenver.edu/~jmandel/data/math4779f21/raws.json
    j = json.load(open(jfile,'r'))
    print('loaded from ',jfile)
    # Take the first station in the boulding box that has data between time_start and time_s2.
    # Then retrieve data for that station between time_start and time_end
    time_start = j['time_start']      # start of data time series
    # time_s2    = j['time_s2']         # end of segment to read coordinates
    time_end  = j['time_end']         # end of data time series
    meso_ts  = j['meso_ts']           # get meso observations time series
    obs_lon =   j['obs_lon']          # where we retrieved observations
    obs_lat =   j['obs_lat']
except:
    print("can't read",jfile,', creating')
    # set up bounds
    time_start = "201806010800"  # June 1 2018 08:00 in format yyyymmddHHMM
    time_s2    = "201806010900"  # June 1 2018 09:00 in format yyyymmddHHMM 
    time_end   = "201907200900"  # June 20 2018 09:00 in format yyyymmddHHMM 
    #time_start=  "201810230100"
    #time_s2=  "201810230300"
    #time_end  =  "201806022300"
    !pip install MesoPy
    from MesoPy import Meso
    bounding_box = "-115, 38, -110, 40"  # min longtitude, latitude
    meso_token="b40cb52cbdef43ef81329b84e8fd874f"       # you should get your own if you do more of this
    m = Meso(meso_token)# create a Meso object
    print('reading MesoWest fuel moisture data')
    json_w(m.variables(),'variables.json')
    meso_obss = m.timeseries(time_start, time_s2, bbox=bounding_box, 
                             showemptystations = '0', vars=vars)   # ask the object for data
    json_w(meso_obss,'meso_obss.json')                        
    # pick one station and retrieve the whole time series.
    station=meso_obss['STATION'][0]
    json_w(station,'station.json')
    lon,lat = (float(station['LONGITUDE']),float(station['LATITUDE']))
    print(station['NAME'],'station',station['STID'],'at',lon,lat)
    e = 0.01   # tolerance
    bb = '%s, %s, %s, %s' % (lon - e, lat - e, lon + e, lat + e)
    print('bounding box',bb)
    meso_ts = m.timeseries(time_start, time_end, bbox=bb, showemptystations = '0', vars=vars)   # ask the object for data
    json_w(meso_ts,'meso_ts.json')                        
    obs_lon, obs_lat = (lon, lat)   # remember station coordinates for later
    j={'time_start':time_start,'time_s2':time_s2,'time_end':time_end,
       'meso_ts':meso_ts,'obs_lon':obs_lon,'obs_lat':obs_lat}
    json_w(j,jfile)
    print('done')

In [ ]:
os.chdir('..')
# process the data retrieved for this station
# print(json.dumps(meso_ts['STATION'][0], indent=4))
from datetime import datetime, timedelta, time
import numpy as np
import matplotlib.pyplot as plt
import pytz
station = meso_ts['STATION'][0]
time_str  = station['OBSERVATIONS']['date_time']
obs_time = [datetime.strptime(t, '%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=pytz.UTC) for t in time_str]
start_time = obs_time[0].replace(minute=0)     # remember obs_time and start_time for later
end_time = obs_time[-1]
obs_data = np.array(station['OBSERVATIONS']["fuel_moisture_set_1"])
# obs_data = np.array(station['OBSERVATIONS']["fuel_moisture"])
# display the data retrieved
#for o_time,o_data in zip (obs_time,obs_data):
#    print(o_time,o_data)
%matplotlib inline
plt.figure(figsize=(16,4))
plt.plot(obs_data,linestyle='-',c='k',label='10-h fuel data')
plt.title(station['STID'] + ' 10 h fuel moisture data')
plt.xlabel('Time (hours)') 
plt.ylabel('Fuel moisture content (%)')
plt.legend()
 

#### 3.2.2 Acquisition of weather data

Our weather data are results from atmospheric models, with assimilated observations from weather stations, satellites, radars, etc. The models can be run in reanalysis mode (for the past, with data for the period modeled)  or in forecast mode (for the future, with only past data assimilated - because future data are not here yet). We use the Real-Time Mesoscale Analysis ([RTMA](https://www.nco.ncep.noaa.gov/pmb/products/rtma/)) interpolated to the RAWS location. RTMA is a real-time product, posted hourly, and available only for few days in the past. We have our own collection of selected RAWS data over past few years, obtained as a side effect of running the fuel moisture modeling software [WRFXPY](https://github.com/openwfm/wrfxpy).

First try to read the data already extracted for this RAWS and staged for download.

In [ ]:
os.chdir('data')
import json
jfile = 'rtma.json'
try:
    ! wget --no-clobber http://math.ucdenver.edu/~jmandel/data/math4779f21/rtma.json
    j = json.load(open(jfile,'r'))
    print('loaded from ',jfile)
    if j['obs_lat']!=obs_lat or j['obs_lon']!=obs_lon:
      print('lon lat doesnot agree, need to load original RTMA files')
      read_rtma=True
    else:
      read_rtma=False
except:
    print("can't read",jfile,', creating')
    read_rtma=True

print('')

Next, functions to get the files, open as grib, and interpolate to the station coordinates

####<font color=red>Note: If read_rtma==True, the notebook will say it crashed when run the first time. This is because it needs to install different version of some python packages and restart runtime. Simply run it again.</fonr>

In [ ]:
# Set up environment to read RTMA gribs
# we will need current numpy for pygrib - needed on Colab, tensorflow is using numpy 1.19\
if read_rtma:
  import subprocess,os
  def load_rtma(path,file,reload=0):
    url='http://math.ucdenver.edu/~jmandel/rtma/' + path 
    if os.path.exists(file):
      if reload:
        print(file + ' already exists, removing')
        os.remove(file)
      else:
        print(file + ' already exists, exiting')
        # add checking size here
        return 0
    try:
      ret = subprocess.check_output(['wget','--no-clobber','--output-document='+ file, url,],stderr=subprocess.STDOUT).decode() # execute command from python strings
      if os.path.exists(file):
        print('loaded ' + url + ' as ' + file)
        return 0
      else: 
        print('file transfer completed, but the file is missing? ' + url)  
      return 1
    except:
      print('file transfer failed: ' + url)
      return 2


Create a function to transfer RTMA files in GRIB2 format from the stash. The function returns zero if the file transfer succeeded. If the file is not available, it returns a nonzero value. Note: if needed, maybe in future add more sophisticated checks, check the return code of wget and if the file size is correct.

In [ ]:
if read_rtma:
  def rtma_grib(t,var):
    tpath = '%4i%02i%02i/%02i' % (t.year, t.month, t.day, t.hour)  # remote path on server
    tstr  = '%4i%02i%02i%02i_' % (t.year, t.month, t.day, t.hour)  # time string for local path
    gribfile = os.path.join('data',tstr + var + '.grib')
    remote = tpath + '/' + var + '.grib'
    if load_rtma(remote,gribfile):
        print('cannot load remote file',remote,'as',gribfile)
        return []
    else:
        try:
            gf=GribFile(gribfile)
            v = np.array(gf[1].values())
        except:
            print('cannot read grib file',gribfile)
            return []
        print('loaded ',gribfile,' containing array shape ',v.shape)
        return gf[1]   # grib message


In [ ]:
if read_rtma:
    times = pd.date_range(start=time_start,end=time_end,freq='1H')
    varnames=['temp','td','precipa']
    j =    read_interp_rtma(varnames,times,obs_lat,obs_lon)      # temperature
    for varname in varnames:
        j[varname]=j[varname].tolist() 
    j['obs_lat']=obs_lat
    j['obs_lon']=obs_lon
    json.dump(j,open('rtma.json','w'),indent=4)
    print('done')

In [ ]:
from scipy.interpolate import LinearNDInterpolator, interpn
from scipy.optimize import root
def interp_to_lat_lon_slow(lats,lons,v,lat,lon): 
    # on mesh with coordinates lats and lons interpolate v to given lat lon
    interp=LinearNDInterpolator(list(zip(lats.flatten(),lons.flatten())),v.flatten())
    return interp(lat,lon)
def interp_to_lat_lon(lats,lons,v,lat,lon):
    # on mesh with coordinates lats and lons interpolate v to given lat lon
    points=(np.array(range(lats.shape[0]),float),np.array(range(lats.shape[1]),float))  # uniform mesh
    def res(ij):  # interpolation of lons lats on the uniform mesh, to noninteger coordinates   
       return np.hstack((interpn(points,lats,ij)-lat, interpn(points,lons,ij)-lon))
    # solve for xi,xj such that lats(xi,xj)=lat lons(xi,xj)=lon, then interpolate to (xi, xj) on uniform grid 
    result = root(res,(0,0)) # solve res(ij) = 0
    if not result.success:
        print(result.message)
        exit(1)
    return interpn(points,v,result.x) 


The interpolation function needs to  be tested.

In [ ]:
def interp_to_lat_lon_test(lats,lons):
    print('testing interp_to_lat_lon')
    vx, vy = np.meshgrid(range(lats.shape[0]),range(lats.shape[1]),indexing='ij')
    i, j = (1,2)
    lat,lon = ((lats[i,j]+lats[i+1,j+1])/2,(lons[i,j]+lons[i+1,j+1])/2)
    vi = interp_to_lat_lon(lats,lons,vx,lat,lon)
    vj = interp_to_lat_lon(lats,lons,vy,lat,lon)
    print(vi,vj,'should be about',i+0.5,j+0.5)
    test_slow = 0
    if test_slow:
        print('Testing against the standard slow method scipy.interpolate.LinearNDInterpolator. Please wait...')
        vi_slow = interp_to_lat_lon_slow(lats,lons,vx,lat,lon)
        print(vi_slow)
        vj_slow = interp_to_lat_lon_slow(lats,lons,vy,lat,lon)
        print(vj_slow)
        
#gf = rtma_grib(start_time,'temp')      #  read the first grib file and use it to test interpolation
#lats, lons = gf.latlons()
#interp_to_lat_lon_test(lats,lons)


In [ ]:
#%debug


Now we are ready for a function to read the RTMA files and interpolate to the station coordinates

In [ ]:
if read_rtma:
  import pandas as pd, json
  def read_interp_rtma(varnames,times,lat,lon):
    # read RTMA from start_time to end_time and interpolate to obs_lat obs_lon
    ntimes = len(times)
    time_str = 'time_str'
    j={time_str:times.strftime('%Y-%m-%d %H:%M').tolist()}
    for varname in varnames:
        j[varname]=np.full(ntimes,np.nan)  # initialize array of nans as list
    n=0
    for t in times:
        tim=t.strftime('%Y-%m-%d %H:%M')
        should_be = j[time_str][n]
        if tim != should_be:
            print('n=',n,'time',tim,'expected',should_be)
            raise 'Invalid time' 
        for varname in varnames:
            gf = rtma_grib(t,varname)   # read and create grib object, download if needed
            if gf:
                lats,lons = gf.latlons()    # coordinates
                v = gf.values()
                vi=interp_to_lat_lon(lats,lons,v,lat,lon) # append to array
                print(varname,'at',t,'interpolated to',lat,lon,' value ',vi)
                j[varname][n] = vi
            else:
                print(varname,'at',t,' could not be loaded')
        n = n+1
    return j

In [ ]:
# %debug


#### 3.2.3 Preprocessing and visualization of the weather data

In [ ]:
rtma = j
td = np.array(rtma['td'])
t2 = np.array(rtma['temp'])
rain=np.array(rtma['precipa'])
# compute relative humidity
rh = 100*np.exp(17.625*243.04*(td - t2) / (243.04 + t2 - 273.15) / (243.0 + td - 273.15))
Ed = 0.924*rh**0.679 + 0.000499*np.exp(0.1*rh) + 0.18*(21.1 + 273.15 - t2)*(1 - np.exp(-0.115*rh))
Ew = 0.618*rh**0.753 + 0.000454*np.exp(0.1*rh) + 0.18*(21.1 + 273.15 - t2)*(1 - np.exp(-0.115*rh))

In [ ]:
%matplotlib inline
plt.figure(figsize=(16,4))
plt.plot(t2,linestyle='-',c='k',label='Temperature')
plt.title(station['STID'] + ' Temperature')
plt.xlabel('Time (hours)') 
plt.ylabel('Temperature (K)')
plt.legend()

In [ ]:
%matplotlib inline
plt.figure(figsize=(16,4))
plt.plot(td,linestyle='-',c='k',label='Dew point')
plt.title(station['STID'] + ' Dew point (K)')
plt.xlabel('Time (hours)') 
plt.ylabel('Dew point (K)')
plt.legend()

In [ ]:
%matplotlib inline
plt.figure(figsize=(16,4))
plt.plot(rh,linestyle='-',c='k',label='Dew point')
plt.title(station['STID'] + ' relative humidity')
plt.xlabel('Time (hours)') 
plt.ylabel('Relative humidity (%)')
plt.legend()

In [ ]:
%matplotlib inline
plt.figure(figsize=(16,4))
plt.plot(Ed,linestyle='-',c='r',label='drying equilibrium')
plt.plot(Ew,linestyle=':',c='b',label='wetting equilibrium')
plt.title(station['STID'] + ' drying and wetting equilibria')
plt.xlabel('Time (hours)') 
plt.ylabel('Fuel moisture contents (%)')
plt.legend()

In [ ]:
%matplotlib inline
plt.figure(figsize=(16,4))
plt.plot(rain,linestyle='-',c='k',label='Precipitation')
plt.title(station['STID'] + ' Precipitation' )
plt.xlabel('Time (hours)') 
plt.ylabel('Precipitation (mm/hour)')
plt.legend()

In [ ]:
print(rain[1900:2000])

Precipitation from RTMA is in kg/m${}^2$. 1m water depth over 1m${}^2$ is 1m${}^3$ with mass 1000 kg thus 1 kg/m${}^2$ is the same as 1 mm of precipitation. RTMA values are accumulations over 1 h so these are values in mm/h. So 9999 mm/h = 10m/h makes no sense. Replace anything over 1m/h by nan and try again.

In [ ]:
rain[rain > 1000] = np.NaN

In [ ]:
%matplotlib inline
plt.figure(figsize=(16,4))
plt.plot(rain,linestyle='-',c='k',label='Precipitation')
plt.title(station['STID'] + ' Precipitation' )
plt.xlabel('Time (hours)') 
plt.ylabel('Precipitation (mm/hour)')
plt.legend()

Fix some missing data, then we can use the data for up to 1942 hours until a biger gap.

In [ ]:
# fix isolated nans
def fixnan(a,n):
    for c in range(n):
        for i in np.where(np.isnan(a)):
            a[i]=0.5*(a[i-1]+a[i+1])
        if not any(np.isnan(a)):
            break
    return a

rain=fixnan(rain,2)
t2=fixnan(t2,2)
rh=fixnan(rh,2)
obs_data=fixnan(obs_data,2)
Ed=fixnan(Ed,2)
Ew=fixnan(Ew,2)

print(np.where(np.isnan(rain)))
print(np.where(np.isnan(t2)))
print(np.where(np.isnan(rh)))
print(np.where(np.isnan(obs_data)))

## 4 Results

### 4.1 Kalman filter with fuel moisture observations, followed by forecasting
We run the model first with Kalman filter for 150 hours. The observations are the RAWS data
After 150 hours, we run in forecast mode - the RAWS data are no longer used, and we run the model from the weather data without the Kalman filter. The weather data are taken to be RTMA interpolated to one RAWS location.
In a real forecasting application, the model would be run from weather forecast rather than data.

In [ ]:
# run KF on an initial data seqment
import numpy as np
import matplotlib.pyplot as plt 

hours=1200 # total simulation
h2 = 300
m = np.zeros(hours) # preallocate
m[0]= obs_data[0]             # initial state  
P = np.zeros(hours)
P[0] = 1e-3 # background state variance
H = np.array([1.])   # all oQ = np.array([0.02]) # process noise variancebserved
Q = np.array([1e-3]) # process noise variance
R = np.array([1e-3]) # data variance
for t in range(hours-1):
    # using lambda construction to pass additional arguments to the model 
    if t < h2 and not np.isnan(obs_data[t]) and not np.isnan(Ew[t]) and not np.isnan(rain[t]): # advance model and run KF
        m[t+1],P[t+1] = ext_kf(m[t],P[t],lambda u: model_moisture(u,Ed[t],Ew[t],rain[t],t,partials=1),Q,
                    d=obs_data[t],H=H,R=R)
    else:  # just advance to next hour, no process noise
        m[t+1],P[t+1] = ext_kf(m[t],P[t],lambda u: model_moisture(u,Ed[t],Ew[t],rain[t],t,partials=1),Q*0.0)

In [ ]:
%matplotlib inline
plt.figure(figsize=(16,4))
plt.plot(Ed[:hours],linestyle='--',c='r',label='Drying Equilibrium')
plt.plot(Ew[:hours],linestyle='--',c='b',label='Wetting Equilibrium')
plt.plot(obs_data[:hours],linestyle=':',c='k',label='RAWS data')
plt.plot(m[:h2],linestyle='-',c='k',label='filtered')
plt.plot(range(h2,hours),m[h2:hours],linestyle='-',c='r',label='forecast')
plt.title(station['STID'] + ' Kalman filtering and forecast with real data')
plt.xlabel('Time (hours)') 
plt.ylabel('Fuel moisture content (%)')
plt.legend()

Clearly, there is a problem - the forecast fuel moisture is too high. We need to assimilate also some parameters of the model, not just its output state. 

### 4.3 Kalman filter on the augmented model

Run augmented filter and plot the result:


In [ ]:
m,Ec = run_augmented_kf(obs_data,Ed,Ew,rain,h2,hours)  # extract from state

In [ ]:
title = station['STID'] +' Kalman filtering and forecast with augmented state, real data. Training 0:%i hmax' % h2
def plot_moisture(hmin,hmax):
  print('training from 0 to',h2,'plot from',hmin,'to',hmax)
  plt.figure(figsize=(16,4))
  plt.plot(range(hmin,hmax),Ed[hmin:hmax],linestyle='--',c='r',label='Drying Equilibrium (%)')
  plt.plot(range(hmin,hmax),Ew[hmin:hmax],linestyle='--',c='b',label='Wetting Equilibrium (%)')
  plt.plot(range(hmin,hmax),Ec[hmin:hmax],linestyle='--',c='g',label='Equilibrium Correction (%)')
  plt.plot(range(hmin,hmax),m[hmin:hmax],linestyle='-',c='k',label='filtered')
  plt.plot(range(hmin,hmax),obs_data[hmin:hmax],linestyle='-',c='b',label='RAWS data (%)')
  plt.plot(range(hmin,hmax),rain[hmin:hmax],linestyle='-',c='b',label='RTMA rain (mm/h)')
  plt.title(title)
  if hmin>=h2:
    plt.plot(m[hmin:h2],linestyle='-',c='k',label='Filtered')
  h1 = np.maximum(hmin,h2)
  plt.plot(range(h1,hmax),m[h1:hmax],linestyle='-',c='r',label='Forecast (%)')
  plt.xlabel('Time (hours)') 
  plt.ylabel('Fuel moisture content (%)')
  plt.legend()

In [ ]:
os.chdir('..')
from data_funcs import to_json, from_json

In [ ]:
os.chdir('data')
kf_orig={'title':title,'hours':hours,'h2':h2,'Ed':Ed,'Ew':Ew,'Ec':Ec,'rain':rain,
            'fm':obs_data,'m':m,'note':'RAWS and RTMA data + m from augmented KF in fmda_kf_rnn_orig'}
to_json(kf_orig,'kf_orig.json')

In [ ]:
plot_moisture(0,hours)

A detailed view of transition from training to forecast:

In [ ]:
plot_moisture(0,600)



In [ ]:
plot_moisture(300,500)

In [ ]:
plot_moisture(300,800)

In [ ]:
plot_moisture(800,1200)

Filtering by extended Kalman filter using RAWS data until 150 hours, then forecasting mode - running the model from interpolated RTMA only. For the first 60 hours the forecast is good, the equilibium correction made the model quite close to data. But then the big spike in equilibrium moisture around 230 hours attracted the solution, and it took a while for it to get back. The spike in the RAWS measurement is there but much smaller. The model becomes inaccurate during periods when the fuel moisture equilibrium is large.

Possible reasons include: 1. There was something in the data we do not know about - maybe it rained but RTMA did not tell us. Try comparing with data from the RAWS itself? 2. The model is too simple, assumes the whole depth of the wood stick is wetting and drying at the same time. Perhaps the moisture got stored in the inside layers of the measurement stick. Try a two-layer model as in van der Kamp (2017) and make the state larger? 

A detailed view of rain episode:

In [ ]:
plot_moisture(900,1100)

It seems there is some rain that the model does not know about.

## RNN for real data, no rain yet

#### Linear modeling by RELU - potential for generalization

In [ ]:
def RELU(x):
  if x>0. :
    return x
  else:
    return 0.

# network computing z = a*x1 + b*x2 with offset c
def linrelu(x,a,b,c):
  y = np.dot(np.array([[a, b], [-a, -b] ]), x) + np.array([c, -c])
  y[0]=RELU(y[0])
  y[1]=RELU(y[1])
  return(np.dot([1,-1],y))-c
x = [1,2]
a = 2
b = 4
c = 3
print(a*x[0]+b*x[1])
linrelu(x,a,b,c)

### Basic RNN on real data 

Try with E average between drying and wetting

In [ ]:
E = (Ed + Ew)/2
print(Ed.shape,Ew.shape,rain.shape)
first_rain=np.nonzero(rain>0)[0][0]
print(first_rain)
hours=first_rain
E=E[:hours]
data=obs_data[:hours]
scale=False

# transform as 2D, (timesteps, features) and (timesteps, outputs)
Et = np.reshape(E,[E.shape[0],1])
datat = np.reshape(data,[data.shape[0],1])
if scale:
  scalerx = MinMaxScaler()
  scalerx.fit(Et)
  Et = scalerx.transform(Et)
  scalery = MinMaxScaler()
  scalery.fit(datat)
  datat = scalery.transform(datat)

Create the model again

In [ ]:
# split data
return_sequences=False
x_train, y_train = staircase(Et,datat,timesteps=5,trainsteps=h2,
                             return_sequences=return_sequences)
print('x_train shape=',x_train.shape)
samples, timesteps, features = x_train.shape
print('y_train shape=',y_train.shape)
# the simplest model possible
activation=['linear','linear']
hidden_units=3
dense_units=1
dense_layers=1
features=1
hours=Et.shape[0]
h0 = tf.convert_to_tensor(datat[:samples],dtype=tf.float32)
# print('initial state=',h0)
# statefull model version for traning
import moisture_rnn
model_fit=moisture_rnn.create_RNN_2(hidden_units=hidden_units, 
                        dense_units=dense_units, 
                        batch_shape=(samples,timesteps,features),
                        stateful=True,
                        return_sequences=return_sequences,
                        # initial_state=h0,
                        activation=activation,
                        dense_layers=dense_layers)
# same model stateless for prediction on the entire dataset - to start onlg
# the real application will switch to prediction after training data end
# and start from the state there
print('model_fit input shape',x_train.shape,'output shape',model_fit(x_train).shape)
from keras.utils.vis_utils import plot_model
plot_model(model_fit, to_file='model_plot.png', 
           show_shapes=True, show_layer_names=True)

In [ ]:
model_predict=create_RNN_2(hidden_units=hidden_units, dense_units=dense_units,  
                        input_shape=(hours,features),stateful = False,
                        return_sequences=True,
                        activation=activation,dense_layers=dense_layers)
# model_predict=create_RNN_sequences(hidden_units=1, dense_units=1, input_shape=(hours,1), 
#                        activation=['linear', 'linear'])
print('model_predict input shape',Et.shape,'output shape',model_predict(Et).shape)
print(model_predict.summary())
from keras.utils.vis_utils import plot_model
plot_model(model_predict, to_file='model_plot.png', 
           show_shapes=True, show_layer_names=True)

In [ ]:
# fitting
DeltaE = 0
w_exact=  [np.array([[1.-np.exp(-0.1)]]), np.array([[np.exp(-0.1)]]), np.array([0.]),np.array([[1.0]]),np.array([-1.*DeltaE])]
w_initial=[np.array([[1.-np.exp(-0.1)]]), np.array([[np.exp(-0.1)]]), np.array([0.]),np.array([[1.0]]),np.array([-1.0])]
w=model_fit.get_weights()
for i in range(len(w)):
  print('weight',i,'shape',w[i].shape,'ndim',w[i].ndim,'given',w_initial[i].shape)
  for j in range(w[i].shape[0]):
    if w[i].ndim==2:
      for k in range(w[i].shape[1]):
        w[i][j][k]=w_initial[i][0][0]/w[i].shape[0]
    else:
      w[i][j]=w_initial[i][0]
model_fit.set_weights(w)
model_fit.fit(x_train, y_train, epochs=5000, verbose=0,batch_size=samples)
w_fitted=model_fit.get_weights()
for i in range(len(w)):
  print('weight',i,' exact:',w_exact[i],':  initial:',w_initial[i],' fitted:',w_fitted[i])

In [ ]:
# evaluate model
model_predict.set_weights(w_fitted)
x_input=np.reshape(Et,(1, hours, 1))
y_output = model_predict.predict(x_input)
print('x_input.shape=',x_input.shape,'y_output.shape=',y_output.shape)
print(shift)
m = np.reshape(y_output,hours)
print('weights=',w)
if scale:
    print('scaling')
    m = scalery.inverse_transform(m)
m = np.reshape(m,hours)
hour=np.array(range(hours))
title="First RNN forecast"
plt.figure(figsize=(16,4))
plt.plot(hour,E,linestyle='--',c='r',label='E=Equilibrium data')
# print(len(hour),len(m_f))
plt.scatter(hour,data,c='b',label='data=10-h fuel data')
if m is not None:
    plt.plot(hour[:h2],m[:h2],linestyle='-',c='k',label='m=filtered')
    plt.plot(hour[h2:hours],m[h2:hours],linestyle='-',c='r',label='m=forecast')
plt.title(title) 
plt.legend()


In [ ]:
# plot subinterval only
def plot_int(lb=0,ub=hours,title="RNN forecast"):
  hour=np.array(range(hours))
  plt.figure(figsize=(16,4))
  plt.plot(hour[lb:ub],E[lb:ub],linestyle='--',c='r',label='Equilibrium data')
  # plt.scatter(hour[lb:ub],data[lb:ub],c='b',label='data=10-h fuel data')
  plt.plot(hour[lb:ub],m[lb:ub],linestyle='-',c='b',label='data=10-h fuel data')
  if lb <= h2:
    ub1 = min(h2,ub)
    plt.plot(hour[lb:ub1],m[lb:ub1],linestyle='-',c='k',label='filtered')
  if ub >= h2:
    lb1 = max(h2,lb)
    plt.plot(hour[lb1:ub],m[lb1:ub],linestyle='-',c='r',label='forecast')
  plt.title(title) 
  plt.legend()

In [ ]:
plot_int()

In [ ]:
plot_int(0,300)

In [ ]:
plot_int(300,500)

In [ ]:
plot_int(500,800)

Next step: two features - drying and wetting equilibria

In [ ]:
print(Ed.shape,Ew.shape,rain.shape)
first_rain=np.nonzero(rain>0)[0][0]
print(first_rain)
hours=first_rain
Ed=Ed[:hours]
Ew=Ew[:hours]
h2 = 300
# print(Ed.shape,Ew.shape)
# (timesteps, features)
Et = np.vstack((Ed, Ew)).T
print(E.shape)
data=obs_data[:hours]

scale=False

# transform as 2D, (timesteps, features) and (timesteps, outputs)
datat = np.reshape(data,[data.shape[0],1])
if scale:
  scalerx = MinMaxScaler()
  scalerx.fit(Et)
  Et = scalerx.transform(Et)
  scalery = MinMaxScaler()
  scalery.fit(datat)
  datat = scalery.transform(datat)

In [ ]:
from utils import hash2

In [ ]:
# Set seed for reproducibility
reproducibility.set_seed()

In [ ]:
# split data
return_sequences=False
x_train, y_train = staircase(Et,datat,timesteps=5,trainsteps=h2,
                             return_sequences=return_sequences)
print('x_train shape=',x_train.shape)
samples, timesteps, features = x_train.shape
print('y_train shape=',y_train.shape)
# the simplest model possible
activation=['linear','linear']
hidden_units=6
dense_units=1
dense_layers=1
features=Et.shape[1]
hours=Et.shape[0]
h0 = tf.convert_to_tensor(datat[:samples],dtype=tf.float32)
# print('initial state=',h0)
# statefull model version for traning
import moisture_rnn
model_fit=moisture_rnn.create_RNN_2(hidden_units=hidden_units, 
                        dense_units=dense_units, 
                        batch_shape=(samples,timesteps,features),
                        stateful=True,
                        return_sequences=return_sequences,
                        # initial_state=h0,
                        activation=activation,
                        dense_layers=dense_layers)
# same model stateless for prediction on the entire dataset - to start onlg
# the real application will switch to prediction after training data end
# and start from the state there
print('model_fit input shape',x_train.shape,'output shape',model_fit(x_train).shape)
print('model_fit input shape',x_train.shape,'output shape',y_train.shape)
from keras.utils.vis_utils import plot_model
plot_model(model_fit, to_file='model_plot.png', 
           show_shapes=True, show_layer_names=True)

In [ ]:
## Check 1: equilibrium input data the same

print(hash2(Et))
print(hash2(x_train))
print(hash2(y_train))

In [ ]:
## Check 2: Untrained RNN initialized with same weights

hash2(model_fit.get_weights())

In [ ]:
model_predict=moisture_rnn.create_RNN_2(hidden_units=hidden_units, dense_units=dense_units,  
                        input_shape=(hours,features),stateful = False,
                        return_sequences=True,
                        activation=activation,dense_layers=dense_layers)
# model_predict=create_RNN_sequences(hidden_units=1, dense_units=1, input_shape=(hours,1), 
#                        activation=['linear', 'linear'])
# print('model_predict input shape',Et.shape,'output shape',model_predict(Et).shape)
print(model_predict.summary())
from keras.utils.vis_utils import plot_model
plot_model(model_predict, to_file='model_plot.png', 
           show_shapes=True, show_layer_names=True)

In [ ]:
## Check 3: Second model initialization same weights

hash2(model_predict.get_weights())

In [ ]:
w_initial=[np.array([[1.-np.exp(-0.1)]]), np.array([[np.exp(-0.1)]]), np.array([0.]),np.array([[1.0]]),np.array([-1.0])]
w=model_fit.get_weights()
for i in range(len(w)):
  print('weight',i,'shape',w[i].shape,'ndim',w[i].ndim,'given',w_initial[i].shape)
  for j in range(w[i].shape[0]):
    if w[i].ndim==2:
      for k in range(w[i].shape[1]):
        w[i][j][k]=w_initial[i][0][0]/w[i].shape[0]
    else:
      w[i][j]=w_initial[i][0]
model_fit.set_weights(w)

In [ ]:
## Check 4: weights the same after this step 

print(hash2(model_fit.get_weights()))
print(hash2(x_train))
print(hash2(y_train))

In [ ]:
# print('model_fit input shape',x_train.shape,'output shape',y_train.shape)
# print('x_train',x_train)
# print('y_train',y_train)

In [ ]:
# reproducibility.set_seed()

In [ ]:
# model_fit.get_weights()

In [ ]:
model_fit.fit(x_train, y_train, epochs=5000, verbose=0,batch_size=samples)
w_fitted=model_fit.get_weights()
for i in range(len(w)):
  print('weight',i,' exact:',w_exact[i],':  initial:',w_initial[i],' fitted:',w_fitted[i])

In [ ]:
## Check 5: Weights NOT the same after fitting

hash2(model_fit.get_weights())

In [ ]:
# evaluate model
model_predict.set_weights(w_fitted)
x_input=np.reshape(Et,(1, hours, 2))
y_output = model_predict.predict(x_input)
print('x_input.shape=',x_input.shape,'y_output.shape=',y_output.shape)
print(shift)
m = np.reshape(y_output,hours)
print('weights=',w)
if scale:
    print('scaling')
    m = scalery.inverse_transform(m)
m = np.reshape(m,hours)
hour=np.array(range(hours))
title="First RNN forecast"
plt.figure(figsize=(16,4))
plt.plot(hour,Ed,linestyle='--',c='r',label='Drying equilibrium')
plt.plot(hour,Ew,linestyle='--',c='b',label='Wetting equilibrium')
# print(len(hour),len(m_f))
plt.scatter(hour,data,c='b',label='data=10-h fuel data')
if m is not None:
    plt.plot(hour[:h2],m[:h2],linestyle='-',c='k',label='m=filtered')
    plt.plot(hour[h2:hours],m[h2:hours],linestyle='-',c='r',label='m=forecast')
plt.title(title) 
plt.legend()


In [ ]:
rnn_orig={'title':'RNN fitting and prediction - original','hours':hours,'h2':h2,'Ed':Ed,'Ew':Ew,'rain':rain,
            'fm':obs_data,'m':m}
# 'w_exact':w_exact,'w_initial':w_initial,'w_fitted':w_fitted
to_json(rnn_orig,'rnn_orig.json')

In [ ]:
hash2(rnn_orig, ['Ed', 'Ew', 'fm', 'rain'])

In [ ]:
import pandas as pd
print(np.sum(pd.util.hash_array(rnn_orig['m'])))
hash2(rnn_orig['m'])

In [ ]:
print(rnn_orig['m'][0])

In [ ]:
print(hash2(rnn_orig['Ed']))
print(rnn_orig['Ed'][0])

In [ ]:
# plot subinterval only
def plot_int(lb=0,ub=hours,title="RNN Prediction"):
  hour=np.array(range(hours))
  plt.figure(figsize=(16,4))
  plt.plot(hour[lb:ub],Ed[lb:ub],linestyle='--',c='r',label='Drying equilibrium')
  plt.plot(hour[lb:ub],Ew[lb:ub],linestyle='--',c='b',label='Wetting equilibrium')
  plt.plot(hour[lb:ub],data[lb:ub],linestyle='-',c='b',label='RAWS fuel moisture data')
  if lb <= h2:
    ub1 = min(h2,ub)
    plt.plot(hour[lb:ub1],m[lb:ub1],linestyle='-',c='k',label='Fuel moisture fitted')
  if ub >= h2:
    lb1 = max(h2,lb)
    plt.plot(hour[lb1:ub],m[lb1:ub],linestyle='-',c='r',label='Fuel moisture prediction')
  plt.title(title) 
  plt.legend()

In [ ]:
plot_int(0,600,title='RNN fitting and prediction')  # again the whole thing

In [ ]:
plot_int(0,300,title='RNN Fitting') 

In [ ]:
plot_int(300,500)

In [ ]:
plot_int(500,800)

In [ ]:
type(model_predict)

### 4.4 A comment on the information flow in the Kalman filter and in neural networks

## 5. Conclusion

We have shown how to combine a model and data for improved forecasting of fuel moisture from weather forecast using the Kalman filter. Augmenting the filter state by a model parameter and joint estimation of augmented state resulted in an improvement of the forecast.

## Contributions of authors 

Not applicable.

## Acknowledgements

This Math Clinic was sponsored by the team of investigators of the NASA grant no. 80NSSC19K1091 *Coupled Interactive Forecasting of Weather, Fire Behavior, and Smoke Impact for Improved Wildland Fire Decision Making* under the NASA ROSES18 Disasters program. The author would like to thank Brian Zhang from the Math Clinic class for bringing the reference van der Kamp et al. (2017) to his attention.

## References

J. Mandel, S. Amram, J. D. Beezley, G. Kelman, A. K. Kochanski, V. Y. Kondratenko, B. H. Lynn, B. Regev, and M. Vejmelka. *Recent advances and applications of WRF-SFIRE.* Natural Hazards and Earth System Science, 14(10):2829–2845, 2014. [doi:10.5194/nhessd-2-1759-2014](https://doi.org/10.5194/nhessd-2-1759-2014)

R. E. Kalman. *A new approach to linear filtering and prediction problems.* Transactions of the ASME – Journal of Basic Engineering, Series D, 82:35–45, 1960. [doi:10.1115/1.3662552](https://doi.org/10.1115/1.3662552)

E. Kalnay. *Atmospheric Modeling, Data Assimilation and Predictability.* Cambridge University Press, 2003. [doi:10.1017/CBO9780511802270](https://doi.org/10.1017/CBO9780511802270)

D. W. van der Kamp, R. D. Moore, and I. G. McKendry. *A model for simulating the moisture content of standardized fuel sticks of various sizes.* Agricultural and Forest Meteorology, 236:123–134, 2017. [doi:10.1016/j.agrformet.2017.01.013](https://doi.org/10.1016/j.agrformet.2017.01.013)

S. F. Schmidt. *Application of state-space methods to navigation problems.* volume 3 of Advances in Control Systems, C. T.  Leondes, ed., pages 293–340. Elsevier, 1966. [doi:10.1016/B978-1-4831-6716-9.50011-4](https://doi.org/10.1016/B978-1-4831-6716-9.50011-4)

M. Vejmelka, A. K. Kochanski, and J. Mandel. *Data assimilation of dead fuel moisture observations from remote automatic weather stations.* International Journal of Wildland Fire, 25:558– 568, 2016. [doi:10.1071/WF14085](https://doi.org/10.1071/WF14085)
